In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('imdb_top_1000.csv',header=0)
df['Released_Year'] = pd.Series(list(x if x.isnumeric() else np.nan for x in df['Released_Year']))
df['Runtime'] = pd.Series(list(x.split()[0] if x.split()[0].isnumeric() else np.nan for x in df['Runtime']))
df['Released_Year'] = df['Released_Year'].astype(float)
df['Runtime'] = df['Runtime'].astype(float)
df['Gross'] = pd.Series(list(x if isinstance(x, float) else float(x.replace(',', '')) for x in df['Gross']))
df = df.dropna(axis=0)
st = ['Released_Year', 'Runtime', 'No_of_Votes', 'Gross', 'IMDB_Rating']
scaler = StandardScaler().fit(df[st])
dfGenre = df['Genre'].str.replace(',','|').str.replace(' ','').str.get_dummies()[['Action', 'Horror', 'Romance', 'Sci-Fi', 'Drama', 'Comedy', 'Animation']]
dfCertificate = [1 if x in ['A', 'R'] else 0 for x in df['Certificate']]
dfRest = df[['Poster_Link', 'Series_Title']]
dfFinal = pd.merge(dfRest, dfGenre, left_index=True, right_index=True)
dfFinal['Certificate'] = dfCertificate
dfFinal[st] = scaler.transform(df[st])
dfFinal.reset_index(drop=True)


,Poster_Link,Series_Title,Action,Horror,Romance,Sci-Fi,Drama,Comedy,Animation,Certificate,Released_Year,Runtime,No_of_Votes,Gross,IMDB_Rating
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,0,0,0,0,1,0,0,1,-0.09343,0.70738,5.61439,-0.43541,4.64978
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,0,0,0,0,1,0,0,1,-1.27716,1.98248,3.57208,0.49239,4.30849
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,1,0,0,0,1,0,0,0,0.65986,1.09377,5.50170,3.97206,3.62591
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,0,0,0,0,1,0,0,1,-1.16955,3.02574,2.18628,-0.18343,3.62591
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,0,0,0,0,1,0,0,0,-2.08426,-1.07003,0.94264,-0.64409,3.62591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,https://m.media-amazon.com/images/M/MV5BYjRmY2...,Giù la testa,0,0,0,0,1,0,0,0,-1.33097,1.28697,-0.92152,-0.67596,-1.15215
709,https://m.media-amazon.com/images/M/MV5BMzAyND...,Kelly's Heroes,0,0,0,0,0,1,0,0,-1.38478,0.78466,-0.87858,-0.67003,-1.15215
710,https://m.media-amazon.com/images/M/MV5BMjAwMT...,The Jungle Book,0,0,0,0,0,0,1,0,-1.54620,-1.76554,-0.53647,0.55223,-1.15215
711,https://m.media-amazon.com/images/M/MV5BZjQyMG...,A Hard Day's Night,0,0,0,0,0,1,0,0,-1.70762,-1.41779,-0.89268,-0.56212,-1.15215


In [4]:
from sklearn.metrics.pairwise import cosine_similarity

def question(df, column):
    css = np.round(cosine_similarity(df.loc[:, ~df.columns.isin(['Poster_Link', 'Series_Title', column])]), 15)
    css = np.where(css < 0, 0, css)
    css = np.where(css == 1, 0, css)
    difference = [[abs(a-b) for a in df[column]] for b in df[column]]
    value = np.multiply(css, difference)
    index = np.argwhere(value == np.max(value))
    return df.iloc[index[0][0]][['Series_Title', column]], df.iloc[index[0][1]][['Series_Title', column]]

column = 'Released_Year'
idcs = question(dfFinal, column)
print(idcs)
# myArray = np.array([((x-np.mean(np.array(myArray)))/np.std(np.array(myArray))) for x in myArray])

(Series_Title     Gully Boy
Released_Year      1.25173
Name: 334, dtype: object, Series_Title     All Quiet on the Western Front
Released_Year                          -3.53703
Name: 461, dtype: object)


In [7]:
genres = [
    "Action",
    "Adventure",
    "Animation",
    "Biography",
    "Comedy",
    #"Crime",
    "Drama",
    #"Family",
    #"Fantasy",
    #"Film-Noir",
    #"History",
    "Horror",
    #"Music",
    #"Musical",
    #"Mystery",
    "Romance",
    "Sci-Fi",
    #"Sport",
    #"Thriller",
    #"War",
    #"Western",
]

def rank(arr):
    res = list()
    while True:
        for i in range(0, len(arr) - 1, 2):
            if input(f"{arr[i]} > {arr[i + 1]}") == "A":
                res.append(arr[i])
            else:
                res.append(arr[i + 1])
        if len(res) > 1 or len(arr) == 3:
            if len(arr) % 2 == 1:
                res.append(arr[-1])
            arr, res = res, list()
        else:
            break
    return res[0]

#def rank(arr):
#    for i in range(len(arr) - 1):
#        if input(f"{arr[i]} > {arr[i + 1]}"):
#            arr[i], arr[i + 1] = arr[i + 1], arr[i]
#    return arr[-1]

def decision(df, column, column_answer): 
    if input() == 'A':
        return df[df[column] > df[column].median()]
    else:
        return df[df[column] < df[column].median()]

#def questionnumeric(df, column):
#    dfq = df[df['No_of_Votes'] > np.percentile(df['No_of_Votes'], 75)]
#    dfq = dfq.reset_index(drop=True)
#    dfqmin = dfq.iloc[dfq[column].idxmin()]['Series_Title']
#    dfqmax = dfq.iloc[dfq[column].idxmax()]['Series_Title']
#    return dfqmin, dfqmax

#Certificate
#Animation

def program(df, genres):
    #dec = decision(df, "A", Runtime)
    genre = rank(genres)
    anim = int(input("animation?")=="A")
    df = df[(df[genre] == 1) & (df["Animation"] == anim)]
    answer_year = question(df, "Released_Year")
    df = decision(df, "Released_Year", answer_year)
    answer_gross = question(df, "Gross")
    df = decision(df, "Gross", answer_gross)
    print(df)

program(dfFinal, genres)

Action > Adventurea
Animation > Biography
Comedy > Drama
Horror > Romance
['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Drama', 'Horror', 'Romance', 'Sci-Fi'] ['Adventure', 'Biography', 'Drama', 'Romance']
['Adventure', 'Biography', 'Drama', 'Romance', 'Sci-Fi'] []
Adventure > Biography
Drama > Romance
['Adventure', 'Biography', 'Drama', 'Romance', 'Sci-Fi'] ['Biography', 'Romance']
['Biography', 'Romance', 'Sci-Fi'] []
Biography > Romance
['Biography', 'Romance', 'Sci-Fi'] ['Romance']
['Romance', 'Sci-Fi'] []
Romance > Sci-Fi
['Romance', 'Sci-Fi'] ['Sci-Fi']
animation?


                                           Poster_Link           Series_Title  \
113  https://m.media-amazon.com/images/M/MV5BMTY3Mj...     A Clockwork Orange   
114  https://m.media-amazon.com/images/M/MV5BMmNlYz...  2001: A Space Odyssey   
271  https://m.media-amazon.com/images/M/MV5BNGViZW...              The Thing   
275  https://m.media-amazon.com/images/M/MV5BNzQzMz...           Blade Runner   
3

In [24]:
df_single_genre = df[~df["Genre"].str.contains(",")].reset_index(drop=True)
df_single_genre

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994.00000,A,142.00000,Drama,9.30000,Two imprisoned men bond over a number of years...,80.00000,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469.00000
1,https://m.media-amazon.com/images/M/MV5BMmEzNT...,Fight Club,1999.00000,A,139.00000,Drama,8.80000,An insomniac office worker and a devil-may-car...,66.00000,David Fincher,Brad Pitt,Edward Norton,Meat Loaf,Zach Grenier,1854740,37030102.00000
2,https://m.media-amazon.com/images/M/MV5BOTQ5ND...,"Il buono, il brutto, il cattivo",1966.00000,A,161.00000,Western,8.80000,A bounty hunting scam joins two men in an unea...,90.00000,Sergio Leone,Clint Eastwood,Eli Wallach,Lee Van Cleef,Aldo Giuffrè,688390,6100000.00000
3,https://m.media-amazon.com/images/M/MV5BZjA0OW...,One Flew Over the Cuckoo's Nest,1975.00000,A,133.00000,Drama,8.70000,A criminal pleads insanity and is admitted to ...,83.00000,Milos Forman,Jack Nicholson,Louise Fletcher,Michael Berryman,Peter Brocco,918088,112000000.00000
4,https://m.media-amazon.com/images/M/MV5BZjA0MT...,American History X,1998.00000,R,119.00000,Drama,8.50000,A former neo-nazi skinhead tries to prevent hi...,62.00000,Tony Kaye,Edward Norton,Edward Furlong,Beverly D'Angelo,Jennifer Lien,1034705,6719864.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,https://m.media-amazon.com/images/M/MV5BMTgxMj...,Mysterious Skin,2004.00000,R,105.00000,Drama,7.60000,A teenage hustler and a young man obsessed wit...,73.00000,Gregg Araki,Brady Corbet,Joseph Gordon-Levitt,Elisabeth Shue,Chase Ellison,65939,697181.00000
72,https://m.media-amazon.com/images/M/MV5BNmE0Yj...,25th Hour,2002.00000,R,135.00000,Drama,7.60000,"Cornered by the DEA, convicted New York drug d...",68.00000,Spike Lee,Edward Norton,Barry Pepper,Philip Seymour Hoffman,Rosario Dawson,169708,13060843.00000
73,https://m.media-amazon.com/images/M/MV5BNDhjMz...,Y tu mamá también,2001.00000,A,106.00000,Drama,7.60000,"In Mexico, two teenage boys and an attractive ...",88.00000,Alfonso Cuarón,Maribel Verdú,Gael García Bernal,Daniel Giménez Cacho,Ana López Mercado,115827,13622333.00000
74,https://m.media-amazon.com/images/M/MV5BMTM5MD...,Dazed and Confused,1993.00000,U,102.00000,Comedy,7.60000,The adventures of high school and junior high ...,78.00000,Richard Linklater,Jason London,Wiley Wiggins,Matthew McConaughey,Rory Cochrane,165465,7993039.00000
